In [1]:
import mysql.connector
from dotenv import load_dotenv
from decouple import config 
import os

In [2]:
load_dotenv()

connection = mysql.connector.connect(
    host= os.getenv("host"),
    user= os.getenv("user"),
    password= os.getenv("password"),
    database="music_trends"
)

cursor = connection.cursor()

In [3]:
music_trends = """CREATE DATABASE "music_trends";"""

In [10]:
artist= """CREATE TABLE artists (
    artist_id INT PRIMARY KEY AUTO_INCREMENT,
    name VARCHAR(255) UNIQUE NOT NULL
);"""
cursor.execute (artist) 

In [11]:
songs="""CREATE TABLE songs (
    song_id INT PRIMARY KEY AUTO_INCREMENT,
    title VARCHAR(255) NOT NULL
);""" 
cursor.execute (songs)

In [13]:
charts = """ CREATE TABLE charts (
    chart_id INT PRIMARY KEY AUTO_INCREMENT,
    song_id INT NOT NULL,
    artist_id INT NOT NULL,
    year INT NOT NULL,
    appearances INT,
    FOREIGN KEY (song_id) REFERENCES songs(song_id) ON DELETE CASCADE,
    FOREIGN KEY (artist_id) REFERENCES artists(artist_id) ON DELETE CASCADE
);
"""
cursor.execute(charts)

In [14]:
genres = """CREATE TABLE genres (
    genre_id INT PRIMARY KEY AUTO_INCREMENT,
    name VARCHAR(100) UNIQUE NOT NULL
);
"""
cursor.execute(genres)

In [15]:
genrestrends = """CREATE TABLE genretrends (
    trend_id INT PRIMARY KEY AUTO_INCREMENT,
    genre_id INT NOT NULL,
    year INT NOT NULL,
    appearances INT,
    FOREIGN KEY (genre_id) REFERENCES genres(genre_id) ON DELETE CASCADE
);
"""
cursor.execute(genrestrends)

In [22]:
import pandas as pd 

generos_df = pd.read_csv("C:\\Users\\Rober\\Desktop\\PROYECTS\\Music Trends Proyect\\CSV\\cleans\\Tendencia_generos.csv")

for row in generos_df.itertuples(index=False):
    cursor.execute(
        """INSERT INTO genres (Year, Genre, Frequency) VALUES (%s, %s, %s)""",
        (row.Year, row.Genre, row.Frecuency)
    )

canciones_df = pd.read_csv("C:\\Users\\Rober\\Desktop\\PROYECTS\\Music Trends Proyect\\CSV\\cleans\\Tendencia_canciones_artistas.csv")

for row in canciones_df.itertuples(index=False):
    cursor.execute(
        """INSERT INTO songs (Song, Artist) VALUES (%s, %s)""",
        (row.Song, row.Artist)

    )

for row in canciones_df.itertuples(index=False):
    cursor.execute(
        """INSERT INTO charts(Date, Position) VALUES (%s, %s)""",
        (row.Date,row.Position)
    )
connection.cursor()
